In [ ]:
import os
from glob import glob
from copy import deepcopy

import numpy as np
import matplotlib.pyplot as plt

import pydicom
from pymedphys.deliverydata import DeliveryData
from pymedphys.dcm import anonymise_dicom, coords_and_dose_from_dcm
from pymedphys.mudensity import find_relevant_control_points, calc_mu_density, get_grid
from pymedphys.logfile import delivery_data_from_logfile
from pymedphys.gamma import gamma_dcm, convert_to_percent_pass, gamma_shell

from pymedphys.plt import pcolormesh_grid

In [ ]:
data_root = r'C:\Users\sbiggs\logfile2dicom\endometrium'

In [ ]:
original = glob(os.path.join(
    data_root, 'original', '*'
))[0]

plan_name = os.path.basename(original)

filepath_out = glob(os.path.join(
    data_root, 'dicom_40ms', plan_name
))[0]

# filepath_out

In [ ]:
single_beam_plan = original = glob(os.path.join(
    data_root, 'single_beam_plan', '*'
))[0]

In [ ]:
original_dcm = pydicom.read_file(single_beam_plan, force=True)

logfile_dcm = pydicom.read_file(filepath_out, force=True)

In [ ]:
# logfile_dcm

In [ ]:
logfile_40ms_dose_dicom_path = glob(os.path.join(
    data_root, 'calced_dose', '*LogfileCalc40msLow*.dcm'
))[0]

logfile_40ms_dose_dicom_path

In [ ]:
# logfile_240ms_dose_dicom_path = glob(os.path.join(
#     data_root, 'calced_dose', '*LogfileCalc240msLow*.dcm'
# ))[0]

# logfile_240ms_dose_dicom_path

In [ ]:
patient_dose_dicom_path = glob(os.path.join(
    data_root, 'calced_dose', '*OneBeam*.dcm'
))[0]

patient_dose_dicom_path

In [ ]:
beam_sequence = logfile_dcm.BeamSequence[0]
control_point = beam_sequence.ControlPointSequence[0]
control_point.IsocenterPosition

In [ ]:
def mudensity_from_dcm(dcm):
    beam_sequence = dcm.BeamSequence[0]
    leaf_boundaries = beam_sequence.BeamLimitingDeviceSequence[-1].LeafPositionBoundaries
    leaf_widths = np.diff(leaf_boundaries)
#     print(leaf_widths)

    assert beam_sequence.BeamLimitingDeviceSequence[-1].NumberOfLeafJawPairs == len(leaf_widths)
    num_leaves = len(leaf_widths)

    control_points = beam_sequence.ControlPointSequence
    num_control_points = len(control_points)

    mlcs = [
        control_point.BeamLimitingDevicePositionSequence[-1].LeafJawPositions
        for control_point in control_points
    ]

    mlcs = [
        np.array([mlc[num_leaves::], -np.array(mlc[0:num_leaves])]).T
        for mlc in mlcs
    ]

    mlcs = np.array(mlcs)
#     print(np.shape(mlcs))

    dicom_jaw = [
        control_point.BeamLimitingDevicePositionSequence[0].LeafJawPositions
        for control_point in control_points
    ]
    
#     print(dicom_jaw)

    jaw = np.array(dicom_jaw)[-1::-1]
    
    second_col = deepcopy(jaw[:,1])
    jaw[:,1] = jaw[:,0]
    jaw[:,0] = second_col
    
    jaw[:,1] = -jaw[:,1]

#     jaw = jaw[None,:] + np.zeros([num_control_points,1])
#     print(np.shape(jaw))
#     print(jaw)

    total_mu = np.array(dcm.FractionGroupSequence[0].ReferencedBeamSequence[0].BeamMeterset)
    final_mu_weight = np.array(beam_sequence.FinalCumulativeMetersetWeight)

    mu = [
        total_mu * np.array(control_point.CumulativeMetersetWeight) / final_mu_weight
        for control_point in control_points
    ]
    mu = np.array(mu)
#     print(np.shape(mu))

    mu_density = calc_mu_density(mu, mlcs, jaw)
    grid = get_grid()
    
    plt.figure(figsize=(10, 8.5))
    plt.pcolormesh(grid['mlc'][-1::-1], grid['jaw'], mu_density)
    plt.colorbar()
    plt.xlabel('MLCX direction (mm)')
    plt.ylabel('ASYMY travel direction (mm)')

    plt.axis('equal')
    plt.show()
    
    return grid, mu_density


grid_original, mu_density_original = mudensity_from_dcm(original_dcm)
grid_logfile, mu_density_logfile = mudensity_from_dcm(logfile_dcm)

In [ ]:
# calc_mu_density?

In [ ]:
# gamma_shell?

In [ ]:
# mu_density_diff = mu_density_logfile - mu_density_original

# max_diff = np.max(np.abs(mu_density_diff))

# plt.figure(figsize=(6, 5.25))
# plt.pcolormesh(grid_original['mlc'][-1::-1], grid_original['jaw'], mu_density_diff, vmin=-max_diff, vmax=max_diff, cmap='bwr')
# cbar = plt.colorbar()
# cbar.set_label('MU Density Difference')
# plt.xlabel('MLCX direction (mm)')
# plt.ylabel('ASYMY travel direction (mm)')

# plt.axis('equal')
# # plt.grid()
# # plt.plot([-150, 150], [70.5, 70.5])
# plt.title('MU Density Difference (Logfile - Original)')

# plt.plot([-150, 150], [68, 68], 'k--', lw=0.5, label='Presented Slice')
# plt.legend()

# plt.show()

In [ ]:
to_be_plotted = np.where(np.abs(coords_reference[2] - 114) < 1)[0]

mu_density_diff = mu_density_logfile - mu_density_original

max_diff = np.max(np.abs(mu_density_diff))

plt.figure(figsize=(6, 5.25))
plt.pcolormesh(grid_original['mlc'][-1::-1], grid_original['jaw'], mu_density_diff, vmin=-max_diff, vmax=max_diff, cmap='bwr')
cbar = plt.colorbar()
cbar.set_label('MU Density Difference')
plt.xlabel('MLCX direction (mm)')
plt.ylabel('ASYMY travel direction (mm)')

plt.axis('equal')
# plt.grid()
# plt.plot([-150, 150], [70.5, 70.5])
plt.title('MU Density Difference (Logfile - Original)')

for i in to_be_plotted:
    coord = coords_reference[2][i] - isocentre[2]
    plt.plot([-150, 150], [coord, coord], 'k--', lw=0.5, label='Presented Slice')
plt.legend()

plt.show()

In [ ]:
mu_max_where = np.where(max_diff == np.abs(mu_density_diff))

grid_original['jaw'][mu_max_where[0]]

In [ ]:
# dose_percent_threshold = 2
# distance_threshold = 2

# gamma = gamma_dcm(
#     logfile_240ms_dose_dicom_path, patient_dose_dicom_path,
#     dose_percent_threshold, distance_threshold, local_gamma=True, random_subset=10000, max_gamma=1.1)

# valid_gamma = gamma[~np.isnan(gamma)]
# print(np.mean(valid_gamma < 1))

# num_points = len(valid_gamma)
# weights = np.ones(num_points)/num_points

# plt.hist(valid_gamma, 20, weights=weights)
# plt.title('Gamma histogram {}% / {} mm | 20% lower dose cutoff'.format(dose_percent_threshold, distance_threshold))
# plt.xlabel('Local Gamma')
# plt.ylabel('Normalised bin count')

# max_x_axis = np.ceil(np.max(valid_gamma))

# plt.xlim([0, max_x_axis])
# plt.show()

In [ ]:
# dose_percent_threshold = 2
# distance_threshold = 2


# gamma = gamma_dcm(
#     logfile_40ms_dose_dicom_path, patient_dose_dicom_path, 
#     dose_percent_threshold, distance_threshold, local_gamma=True)

# valid_gamma = gamma[~np.isnan(gamma)]
# print(np.mean(valid_gamma < 1))

# num_points = len(valid_gamma)
# weights = np.ones(num_points)/num_points

# plt.hist(valid_gamma, 20, weights=weights)
# plt.title('Gamma histogram {}% / {} mm | 20% lower dose cutoff'.format(dose_percent_threshold, distance_threshold))
# plt.xlabel('Local Gamma')
# plt.ylabel('Normalised bin count')

# max_x_axis = np.ceil(np.max(valid_gamma))

# plt.xlim([0, max_x_axis])
# plt.show()

In [ ]:
coords_reference, dose_reference = coords_and_dose_from_dcm(
    patient_dose_dicom_path)
# coords_evaluation_240, dose_evaluation_240 = coords_and_dose_from_dcm(
#     logfile_240ms_dose_dicom_path)

coords_evaluation_40, dose_evaluation_40 = coords_and_dose_from_dcm(
    logfile_40ms_dose_dicom_path)

In [ ]:
np.where(np.abs(coords_reference[2] - 70.5) < 10)[0]

In [ ]:
keep = dose_reference > 0.2*np.max(dose_reference)
num_points = np.sum(keep)
weights = np.ones(num_points)/num_points

# diff_240 = (dose_evaluation_240 - dose_reference)[keep]
diff_40 = (dose_evaluation_40 - dose_reference)[keep]

In [ ]:
# plt.hist(diff_40, 100, weights=weights)
# np.std(diff_40)

In [ ]:
# plt.hist(diff_240, 100, weights=weights)
# np.std(diff_240)

In [ ]:
dose_percent_threshold = 1
distance_threshold = 1
lower_percent_dose_cutoff = 5

gamma = gamma_dcm(
    patient_dose_dicom_path, logfile_40ms_dose_dicom_path,
    dose_percent_threshold, distance_threshold, local_gamma=True, random_subset=None, lower_percent_dose_cutoff=lower_percent_dose_cutoff)

valid_gamma = gamma[~np.isnan(gamma)]
print(np.mean(valid_gamma < 1))

num_points = len(valid_gamma)
weights = np.ones(num_points)/num_points

plt.hist(valid_gamma, 20, weights=weights)
plt.title('Gamma histogram {}% / {} mm | {}% lower dose cutoff'.format(dose_percent_threshold, distance_threshold, lower_percent_dose_cutoff))
plt.xlabel('Local Gamma')
plt.ylabel('Normalised bin count')

max_x_axis = np.ceil(np.max(valid_gamma))

plt.xlim([0, max_x_axis])
plt.show()

In [ ]:
# to_be_plotted = np.where(np.abs(coords_reference[2] - 114) < 1)[0]

# mu_density_diff = mu_density_logfile - mu_density_original



plt.show()

In [ ]:
# mu_density_diff

In [ ]:

fig = plt.figure(figsize=(6.12,5))

ax1 = plt.gca()
# ax2 = plt.subplot(122)


max_diff = np.max(np.abs(mu_density_diff))

# plt.figure(figsize=(6, 5.25))
mesh = ax1.pcolormesh(grid_original['mlc'][-1::-1], grid_original['jaw'], mu_density_diff, vmin=-max_diff, vmax=max_diff, cmap='bwr')
cbar = fig.colorbar(mesh, ax=ax1)
cbar.set_label('MU density difference')
ax1.set_xlabel('MLCX direction (mm)')
ax1.set_ylabel('ASYMY travel direction (mm)')

ax1.axis('equal')
# plt.grid()
# plt.plot([-150, 150], [70.5, 70.5])
ax1.set_title('MU density difference map (logfile - planned)')

for i in to_be_plotted:
    coord = coords_reference[2][i] - isocentre[2]
    ax1.plot([-150, 150], [coord, coord], 'k--', lw=0.5, label='Presented slice')
ax1.legend()

plt.tight_layout()

fig.savefig('figure1.png', dpi=300)


# num_points = len(valid_gamma)
# weights = np.ones(num_points)/num_points

# pass_rate = np.round(np.mean(valid_gamma < 1) * 100, decimals=1)



# ax2.hist(valid_gamma, 40, weights=weights, label="Gamma pass = {}%".format(pass_rate))
# ax2.set_title('Local gamma histogram {}% / {} mm | {}% lower dose cutoff'.format(dose_percent_threshold, distance_threshold, lower_percent_dose_cutoff))
# ax2.set_xlabel('Local Gamma')
# ax2.set_ylabel('Normalised bin count')

# max_x_axis = np.ceil(np.max(valid_gamma))

# ax2.set_xlim([0, 2])
# ax2.legend()
# plt.show()

In [ ]:
beam_sequence = logfile_dcm.BeamSequence[0]
control_point = beam_sequence.ControlPointSequence[0]
isocentre = np.array(control_point.IsocenterPosition)

In [ ]:
# # to_be_plotted = np.where(np.abs(coords_reference[2] - 70.5) < 60)[0]
# to_be_plotted = np.where(np.abs(coords_reference[2] - 114) < 1)[0]

# mu_density_diff = mu_density_logfile - mu_density_original

# max_diff = np.max(np.abs(mu_density_diff))

# # plt.figure(figsize=(5, 4.25))
# plt.figure(figsize=(10, 9.5))
# plt.pcolormesh(grid_original['mlc'][-1::-1], grid_original['jaw'], mu_density_diff, vmin=-max_diff, vmax=max_diff, cmap='bwr')
# cbar = plt.colorbar()
# cbar.set_label('MU Density Difference')
# plt.xlabel('MLCX direction (mm)')
# plt.ylabel('ASYMY travel direction (mm)')

# plt.axis('equal')
# # plt.grid()
# # plt.plot([-150, 150], [70.5, 70.5])
# plt.title('MU Density Difference (Logfile - Original)')

# for i in to_be_plotted:
#     coord = coords_reference[2][i] - isocentre[2]
#     plt.plot([-150, 150], [coord, coord], 'k', lw=0.5, label='Presented Slice')
# # plt.legend()

# plt.show()


In [ ]:
# https://stackoverflow.com/questions/35710931/remove-a-section-of-a-colormap

# import numpy as np
# import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# Remove the middle 40% of the RdBu_r colormap
interval = np.hstack([np.linspace(0, 0.35), np.linspace(0.65, 1)])
colors = plt.cm.coolwarm(interval)
cmap = LinearSegmentedColormap.from_list('name', colors)

# Plot a comparison of the two colormaps
fig, axes = plt.subplots(ncols=2)
data = np.random.random((10, 10))

im = axes[0].imshow(data, cmap=plt.cm.RdBu_r, vmin=0, vmax=1)
fig.colorbar(im, ax=axes[0], orientation='horizontal', ticks=[0, 0.5, 1])
axes[0].set(title='Original Colormap')

im = axes[1].imshow(data, cmap=cmap, vmin=0, vmax=1)
fig.colorbar(im, ax=axes[1], orientation='horizontal', ticks=[0, 0.5, 1])
axes[1].set(title='New Colormap')

plt.show()

In [ ]:
np.max(np.abs(diff[:,:,i])) / np.max([np.max(dose_evaluation_40), np.max(dose_reference)])

In [ ]:
lower_percent_dose_cutoff

new_dose_cutoff = 20 * 0.7603567305000001 / 100
make_nan = dose_evaluation_40 < new_dose_cutoff

new_gamma = deepcopy(gamma)
new_gamma[make_nan] = np.nan


In [ ]:
i = to_be_plotted[0]
diff = dose_evaluation_40 - dose_reference
# max_diff = np.max(np.abs(diff))
# max_dose = np.max([np.max(dose_evaluation_40), np.max(dose_reference)])

max_diff = np.max(np.abs(diff[:,:,i]))
max_dose = np.max([np.max(dose_evaluation_40[:,:,i]), np.max(dose_reference[:,:,i])])

y, x = pcolormesh_grid(coords_reference[0], coords_reference[1])

fig = plt.figure(figsize=(15,8.2))

ax1 = plt.subplot(221, aspect='equal')
ax2 = plt.subplot(222, sharex=ax1, sharey=ax1, aspect='equal')
ax3 = plt.subplot(223, sharex=ax1, sharey=ax1, aspect='equal')
ax4 = plt.subplot(224, sharex=ax1, sharey=ax1, aspect='equal')


mesh1 = ax1.pcolormesh(x, y, dose_reference[:,:,i], vmin=0, vmax=max_dose)
cbar1 = fig.colorbar(mesh1, ax=ax1)
cbar1.set_label('Single fraction dose (Gy)')
ax1.invert_yaxis()
ax1.set_title('Planned dose')
ax1.set_xlabel('x (mm)')
ax1.set_ylabel('z (mm)')



mesh2 = ax2.pcolormesh(x, y, dose_evaluation_40[:,:,i], vmin=0, vmax=max_dose)

cbar2 = fig.colorbar(mesh2, ax=ax2)
cbar2.set_label('Single fraction dose (Gy)')

ax2.set_title('Logfile generated dose')
ax2.set_xlabel('x (mm)')
ax2.set_ylabel('z (mm)')



mesh3 = ax3.pcolormesh(x, y, diff[:,:,i], vmin=-max_diff, vmax=max_diff, cmap='bwr')

cbar3 = plt.colorbar(mesh3, ax=ax3)
cbar3.set_label('Single fraction dose difference (Gy)')

ax3.set_title('Dose difference (logfile - planned)')
ax3.set_xlabel('x (mm)')
ax3.set_ylabel('z (mm)')



mesh4 = ax4.pcolormesh(x, y, new_gamma[:,:,i], vmin=0, vmax=2, cmap=cmap)

cbar = fig.colorbar(mesh4, ax=ax4)
cbar.set_label('Gamma')

ax4.set_title('Local gamma distribution {}% / {} mm'.format(dose_percent_threshold, distance_threshold))
ax4.set_xlabel('x (mm)')
ax4.set_ylabel('z (mm)')

plt.tight_layout()

fig.savefig('figure2.png', dpi=300)


In [ ]:


valid_gamma = new_gamma[:,:,i][~np.isnan(new_gamma[:,:,i])]
print(np.mean(valid_gamma < 1))

In [ ]:
# np.where(np.abs(diff) == max_diff)

In [ ]:


# i = 54

# nanned_dose_evaluation == 0




def create_plots(i):
    print(i)
    

    
    

    
    
    

    

for i in to_be_plotted:
    print(coords_reference[2][i])
    create_plots(i)

# for i in np.where(np.abs(coords_reference[2] - 68) < 1)[0]:
#     print(coords_reference[2][i])
#     create_plots(i)